In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from multiprocessing import Pool
import time
import datetime
import numpy as np
from multiprocessing import Pool
import warnings
warnings.filterwarnings('ignore')

## Load data

In [43]:
listings_df = pd.read_csv("listings.csv", dtype='unicode')
calendar_df = pd.read_csv("calendar.csv", dtype='unicode')

## View Data

In [44]:
listings_df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,11551,https://www.airbnb.com/rooms/11551,20191105115249,2019-11-06,Arty and Bright London Apartment in Zone 2,Unlike most rental apartments out there my fla...,"Amenities Bedding: 1 Double bed, 1 living room...",Unlike most rental apartments out there my fla...,family,Not even 10 minutes by metro from Victoria Sta...,...,t,f,strict_14_with_grace_period,f,t,2,2,0,0,1.58
1,38151,https://www.airbnb.com/rooms/38151,20191105115249,2019-11-06,Double room/ lounge,NaN,"Comfortable, large double room /lounge area av...","Comfortable, large double room /lounge area av...",none,NaN,...,f,f,flexible,f,f,1,0,1,0,NaN
2,13913,https://www.airbnb.com/rooms/13913,20191105115249,2019-11-06,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,"Hello Everyone, I'm offering my lovely double ...",My bright double bedroom with a large window h...,business,Finsbury Park is a friendly melting pot commun...,...,f,f,moderate,f,f,2,1,1,0,0.17
3,38407,https://www.airbnb.com/rooms/38407,20191105115249,2019-11-06,Canary Wharf Immaculate Apt for 2,"The bright, light and stylish apartment in Can...","An entire bright, light and stylish apartment....","The bright, light and stylish apartment in Can...",none,Very easy to get to all the main sites. The tu...,...,t,f,strict_14_with_grace_period,f,f,1,1,0,0,1.23
4,90700,https://www.airbnb.com/rooms/90700,20191105115249,2019-11-06,Sunny Notting Hill flat & terrace,This is a home not a hotel - for the cost gues...,This charming 1 bedroom with en-suite bathroom...,This is a home not a hotel - for the cost gues...,none,A quick guide or a run through about the area ...,...,t,f,strict_14_with_grace_period,f,f,2,2,0,0,3.33


In [45]:
calendar_df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,11551,2019-11-06,f,$105.00,$105.00,3,180
1,90700,2019-11-06,f,$125.00,$125.00,90,1125
2,90700,2019-11-07,f,$125.00,$125.00,90,1125
3,90700,2019-11-08,f,$125.00,$125.00,90,1125
4,90700,2019-11-09,f,$125.00,$125.00,90,1125


In [46]:
anim_cal_df = calendar_df[['listing_id', 'date', 'price', 'adjusted_price']]

In [47]:
anim_listing_df = listings_df[['id','latitude','longitude']]

In [79]:
anim_df = anim_cal_df.merge(anim_listing_df, left_on='listing_id', right_on='id')

In [80]:
anim_df.head()

,listing_id,date,price,adjusted_price,id,latitude,longitude
0,11551,2019-11-06,$105.00,$105.00,11551,51.46225,-0.11732
1,11551,2019-11-07,$105.00,$105.00,11551,51.46225,-0.11732
2,11551,2019-11-08,$105.00,$105.00,11551,51.46225,-0.11732
3,11551,2019-11-09,$105.00,$105.00,11551,51.46225,-0.11732
4,11551,2019-11-10,$105.00,$105.00,11551,51.46225,-0.11732


In [81]:
test_df = anim_df[0:5000]

In [82]:
test_df['price'] = test_df['price'].astype(str)

In [28]:
p = Pool(processes=4)

In [30]:
def f(x):
    return x.replace("$","")

result = p.map_async(f, anim_df['price'])
anim_df['price'] = result.get()

p.close
p.terminate

<bound method Pool.terminate of <multiprocessing.pool.Pool object at 0x2e538a150>>

In [ ]:
# def f(x):
#     return x.replace("-","")

# p = Pool(processes=4)
# result = p.map_async(f, anim_df['date'])
# anim_df['date'] = result.get()

In [ ]:
# for i,val in enumerate(test_df['date']):
#     test_df['date'][i] = time.mktime(datetime.datetime.strptime(test_df['date'][i], "%Y-%m-%d").timetuple())

In [83]:
test_df['date'] = test_df['date'].astype(str)

In [84]:
len_array = test_df.shape[0]

split1 = round(len_array/4)

splits = [0, split1, split1*2, split1*3, len_array]

chunks = [test_df['date'][splits[0]:splits[1]],
test_df['date'][splits[1]:splits[2]],
test_df['date'][splits[2]:splits[3]],
test_df['date'][splits[3]:splits[4]]]

print(splits)

[0, 1250, 2500, 3750, 5000]


In [97]:
def f(list):
    return_list = []
    for i, val in enumerate(list):
        return_list.append(time.mktime(datetime.datetime.strptime(test_df['date'][i], "%Y-%m-%d").timetuple()))
        
    print('hi')
    return return_list


result = p.map_async(f, chunks)
result_list = result.get()


In [95]:
all_results = result_list[0] + result_list[1] + result_list[2] + result_list[3]
test_df['date'] = all_results

In [92]:
result_list[0]

0       2019-11-06
1       2019-11-07
2       2019-11-08
3       2019-11-09
4       2019-11-10
           ...    
1245    2020-04-04
1246    2020-04-05
1247    2020-04-06
1248    2020-04-07
1249    2020-04-08
Name: date, Length: 1250, dtype: object

In [89]:
test_df.head(30)

,listing_id,date,price,adjusted_price,id,latitude,longitude
0,11551,NaN,$105.00,$105.00,11551,51.46225,-0.11732
1,11551,NaN,$105.00,$105.00,11551,51.46225,-0.11732
2,11551,NaN,$105.00,$105.00,11551,51.46225,-0.11732
3,11551,NaN,$105.00,$105.00,11551,51.46225,-0.11732
4,11551,NaN,$105.00,$105.00,11551,51.46225,-0.11732
5,11551,NaN,$105.00,$105.00,11551,51.46225,-0.11732
6,11551,NaN,$105.00,$105.00,11551,51.46225,-0.11732
7,11551,NaN,$105.00,$105.00,11551,51.46225,-0.11732
8,11551,NaN,$105.00,$105.00,11551,51.46225,-0.11732
9,11551,NaN,$105.00,$105.00,11551,51.46225,-0.11732


In [ ]:
test_df.to_csv('animation.csv')

In [ ]:
import time
from multiprocessing import Pool
 
def f(a_list):
    out = 0
    for n in a_list:
        out += n*n
        time.sleep(0.1)
 
    return out
 
def f_amp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    pool = Pool(processes=5)
 
    result = pool.map_async(f, chunks)
 
    while not result.ready():
        print("Running...")
        time.sleep(0.5)
 
    return sum(result.get())

a = f_amp([1,12,2,2,2,2,2,2,2,2,2,2,2,2])

print(a)

In [ ]:
[a_list[i::5] for i in range(5)]